In [ ]:
# Before we begin, run this cell if you are using Colab
!git clone -b 3-ysi-tutorial https://github.com/nestauk/im-tutorials.git

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

## Exercise 1: Scrape the main table of UK's Yearly Box Office using BeautifulSoup

In [ ]:
url = 'https://www.boxofficemojo.com/intl/uk/yearly/'

In [ ]:
# Access the webpage content
r = requests.get(url)

In [ ]:
# Parse the HTML page
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
# Choose the relevant table
table = soup.find_all('table')[4]

In [ ]:
# Parse and store the data of every table row
lst = []
for row in table.find_all('tr'):
    s = pd.Series([data.text for data in row.find_all('td')])
    lst.append(s)

In [ ]:
# Concatenate the data in a Pandas DataFrame and place the first row of the DataFrame as header. Use the .head() method to check your DataFrame.
data = pd.concat(lst, axis=1).T

# Grab the first row for the header
new_header = data.iloc[0]

# Take the data less the header row
data = data[1:]

# Set the header row as the df header
data.columns = new_header

data.head()

In [ ]:
# Q1
data.Distributor.value_counts().plot(kind='bar', title='Films per distributor')
plt.show()

In [ ]:
# Q2

# Remove the £ symbol and transform strings to integers
data['Gross'] = data['Gross'].apply(lambda x: int(x[1:].replace(',', '')))

# Group the data by Distributor and add the Gross value of their movies
data.groupby('Distributor')['Gross'].sum().sort_values(ascending=False).plot(kind='bar', title='Gross earning by distributor')

plt.show()

## Exercise 2: Use Selenium to scrape Box Office Mojo's top #100 for every year between 2002 and 2019.

In [ ]:
# # RUN THIS CELL WHEN USING THE NOTEBOOK LOCALLY - YOU SHOULD INSTALL SELENIUM FIRST
# import selenium.webdriver
# # Path to the Chrome driver for my Mac -- yours will differ
# mac_path = '../../chromedriver'
# driver = selenium.webdriver.Chrome(executable_path=mac_path)

In [ ]:
# # RUN THIS CELL WHEN USING THE NOTEBOOK ON COLAB - NO PREVIOUS INSTALLATION OF SELENIUM IS NEEDED
# # install chromium, its driver, and selenium
# !apt update
# !apt install chromium-chromedriver
# !pip install selenium
# # set options to be headless
# from selenium import webdriver
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# # open it, go to a website, and get results
# driver = webdriver.Chrome('chromedriver',options=options)

In [ ]:
def html2df(source, q):
    """A wrapper of the scraping pipeline we used before."""
    # Parse the HTML page
    soup = BeautifulSoup(source, 'html.parser')

    # Choose the relevant table
    table = soup.find_all('table')[4]

    # Parse and store the data of every table row
    lst = []
    for row in table.find_all('tr'):
        s = pd.Series([data.text for data in row.find_all('td')])
        lst.append(s)

    # Concatenate the data in a Pandas DataFrame and place the first row of the DataFrame as header.
    data = pd.concat(lst, axis=1).T

    # Grab the first row for the header
    new_header = data.iloc[0]

    # Take the data less the header row
    data = data[1:]

    # Set the header row as the df header
    data.columns = new_header
    
    # Add a new column tagging the page we scraped
    data['page'] = q 
    
    return data

In [ ]:
url = 'https://www.boxofficemojo.com/intl/uk/yearly/'

In [ ]:
# URL to use in Selenium
driver.get(url)

lst = []
# Loop over the years.
for i in range(2001, 2019):
    year = str(i + 1)
    # Locate Hyperlinks by partial link text
    elem = driver.find_element_by_partial_link_text(year)
    # Click on the next page
    elem.click()
    # Store the Pandas DataFrame with the scraped content in a list
    lst.append(html2df(driver.page_source, year))

# Concatenate all Pandas DataFrames
annual_top_100 = pd.concat(lst, sort=False)

In [ ]:
annual_top_100.sample(10)

In [ ]:
print(f'% OF MISSING VALUES PER COLUMN\n{(annual_top_100.isnull().sum() / annual_top_100.shape[0]) * 100}')

In [ ]:
print(f'(MOVIES, COLUMNS) -> {annual_top_100.shape}')